# Arrhythmic Cardiac Cine MRI

In [ ]:
using KomaMRI, PlotlyJS, Plots, Printf # hide

include(joinpath(dirname(pathof(KomaMRI)), "../examples/3.tutorials/utils/RRVariability.jl")) # hide

sys = Scanner() # hide

This tutorial shows how to simulate cardiac cine MRI using Koma.
including cases with variable RR intervals (i.e., arrhythmias). You'll learn how to:

1. Simulate a clean cine acquisition with constant RR intervals.
2. Introduce arrhythmias (variable RR) in the cardiac phantom.
3. Observe how this desynchronization degrades image quality.
4. Fix the acquisition by synchronizing the sequence with the phantom's RR variability (manual triggering).

### 1. Constant RR for Phantom and Sequence

We will begin by simulating a cardiac cine on a myocardial phantom with a constant RR interval.
Let's call the `heart_phantom` function to create a ring-shaped phantom filled with blood, which resembles the left ventricle:

In [ ]:
obj = heart_phantom()

By default, this phantom exhibits periodic contraction and rotation, with a period of 1 second:

In [ ]:
p1 = plot_phantom_map(obj, :T1 ; height=450, time_samples=21) # hide

Now, we will create a bSSFP cine sequence with the following parameters:

In [ ]:
RRs          = [1.0]       # [s] constant RR interval
N_matrix     = 50          # image size = N x N
N_phases     = 40          # Number of cardiac phases
FOV          = 0.11        # [m]
TR           = 20e-3       # [s]
flip_angle   = 10          # [º]
adc_duration = 0.2e-3

In [ ]:
seq = bSSFP_cine(
    FOV, N_matrix, TR, flip_angle, RRs, N_phases, sys;
    N_dummy_cycles = 40, adc_duration = adc_duration,
)

# Simulation  # hide
raw1 = simulate(obj, seq, sys) # hide
# Reconstruction # hide
frames1 = reconstruct_cine(raw1, seq, N_matrix, N_phases) # hide

The simulation and subsequent reconstruction produces the following cine frames,
which look clean and temporally coherent:

In [ ]:
fps = 25 # hide
p2 = plot_cine(frames1, fps; Δt=TR, filename="../assets/tut-7-frames1.gif"); # hide
display(p2)

### 2. Arrhythmic Phantom: Variable RR, Constant Sequence

Now, we will introduce arrhythmias in the phantom by modifying its RR intervals.
However, we will keep the sequence with a constant RR interval of 1 second:

In [ ]:
RRs = [900, 1100, 1000, 1000, 1000, 800] .* 1e-3


# Apply the new RRs to the phantom (both contraction and rotation):
obj.motion.motions[1].time.periods = RRs
obj.motion.motions[2].time.periods = RRs

Since the sequence still assumes a constant RR, it becomes unsynchronized with the phantom.
This produces artifacts and temporal inconsistencies in the cine.

In [ ]:
# Simulation  # hide
raw2 = simulate(obj, seq, sys) # hide
# Reconstruction # hide
frames2 = reconstruct_cine(raw2, seq, N_matrix, N_phases) # hide

plot_cine(frames2, fps; Δt=TR, filename="tut-7-frames2.gif")

### 3. Prospective Triggering: Resynchronized Acquisition
To correct this, we synchronize the sequence by providing it the same RR variability as the phantom:

In [ ]:
seq = bSSFP_cine(
    FOV, N_matrix, TR, flip_angle, RRs, N_phases, sys;
    N_dummy_cycles = 40, adc_duration = adc_duration,
)

This manually mimics cardiac triggering. The resulting cine is
once again correctly aligned, despite the underlying arrhythmia.

In [ ]:
# Simulation  # hide
raw3 = simulate(obj, seq, sys) # hide
# Reconstruction # hide
frames3 = reconstruct_cine(raw3, seq, N_matrix, N_phases) # hide

plot_cine(frames3, fps; Δt=TR, filename="tut-7-frames3.gif")

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*